<h2>Import packages </h2>



In [1]:
#required for manipulating data
import pandas as pd
import numpy as np

#enable Google drive API
import gspread

#required for building the interactive dashboard
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

<h2>Connect to bank data and cleanup your transactions</h2>



In [2]:
gc = gspread.service_account(filename="service_account.json")
sh = gc.open("revolut")

In [3]:
ws = sh.worksheet('Sheet1')
df = pd.DataFrame(ws.get_all_records())
df.head()

,Type,Product,Started Date,Completed Date,Description,Amount,Fee,Currency,State,Balance
0,CARD_PAYMENT,Current,2023-01-26 22:02:47,2023-01-27 10:10:12,Tesco Stores 6601,-6.35,0.0,USD,COMPLETED,521.07
1,CARD_PAYMENT,Current,2023-01-26 16:13:50,2023-01-27 11:50:32,Zettle_*donovan?s Bake,-2.50,0.0,USD,COMPLETED,518.57
2,CARD_PAYMENT,Current,2023-01-26 8:26:35,2023-01-27 13:42:55,apple.com/bill,-3.99,0.0,USD,COMPLETED,514.58
3,CARD_PAYMENT,Current,2023-01-27 13:16:59,2023-01-28 9:59:20,Tesco Stores 6601,-4.85,0.0,USD,COMPLETED,509.73
4,CARD_PAYMENT,Current,2023-01-27 10:43:22,2023-01-28 11:48:36,Zettle_*the Good Eatin,-3.30,0.0,USD,COMPLETED,506.43


<h2>Clean the data</h2>

In [4]:
df = df[['Completed Date','Description','Amount']] #keeping desired data
df['Description'] = df['Description'].map(str.lower)

df = df.rename(columns={'Completed Date': 'Date'})
df['Category'] = 'unassigned'

df.head()

,Date,Description,Amount,Category
0,2023-01-27 10:10:12,tesco stores 6601,-6.35,unassigned
1,2023-01-27 11:50:32,zettle_*donovan?s bake,-2.50,unassigned
2,2023-01-27 13:42:55,apple.com/bill,-3.99,unassigned
3,2023-01-28 9:59:20,tesco stores 6601,-4.85,unassigned
4,2023-01-28 11:48:36,zettle_*the good eatin,-3.30,unassigned


In [21]:
#Assign transactions to the correct category

# Self-Care

df['Category'] = np.where(df['Description'].str.contains(
    'cash at tesco old st h exp|boots|royal'), 
    'Self-Care', df['Category'] )
    
# Fines

df['Category'] = np.where(df['Description'].str.contains(
    'car rental'), 
    'Fines', df['Category'] )
    
# Lore So What

df['Category'] = np.where(df['Description'].str.contains(
    'tubebuddy|itunes|dario|calendly|canva|epidemic|upwork|lada'), 
    'Study', df['Category'] )
    
# Coffee

df['Category'] = np.where(df['Description'].str.contains(
    'lavelle|hart|starbucks|barista|new road|mama shelter'), 
    'Coffee', df['Category'] )
    
# Shopping
    
df['Category'] = np.where(df['Description'].str.contains(
    'islington|at camden town'), 
    'Shopping', df['Category'] )
    
# Restaurants

df['Category'] = np.where(df['Description'].str.contains(
    'bakehouse|zettle|caravan|kod|eating|o ver|mcdonald|manteca|wine house|giacomo|real greek|restaurant|katsute|tonkotsu|zia lucia|viet|change please|me zhi chua|osm'), 
    'Restaurants', df['Category'] )
        
# Entertainment
    
df['Category'] = np.where(df['Description'].str.contains(
    'montys|urban|oshveda|egg|francesco|budgens whitechapel'), 
    'Entertainment', df['Category'] )
    
# Gifts
    
df['Category'] = np.where(df['Description'].str.contains(
    'gucci|blomma'), 
    'Gifts', df['Category'] )
    
# Services
    
df['Category'] = np.where(df['Description'].str.contains(
    'apple|snappy|exchanged to usd'), 
    'Services', df['Category'] )
    
# Excluded
    
df['Category'] = np.where(df['Description'].str.contains(
    'from|paypal|amznmktplace|starnow|refund|giffgaff|backstage|hectagon|tower hamlets bc|sweet suites|temporary hold|cm.com'), 
    'Excluded', df['Category'] )

# Groceries

df['Category'] = np.where(df['Description'].str.contains(
    'tesco|sainsbury|asda|lidl|toogoodtog|nisa|market|millennium mini store'), 
    'Groceries', df['Category'] )

# Transport
    
df['Category'] = np.where(df['Description'].str.contains(
    'uber|zipcar|bird|tfl|Ewa'), 
    'Transport', df['Category'] )
    
# Travel
    
df['Category'] = np.where(df['Description'].str.contains(
    'ryanair|easyjet|airways'), 
    'Travel', df['Category'] )

# Convert the "Date" column to a datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract the month and year information
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
    
pd.options.display.max_rows = 999
df.head(200)

,Date,Category,Description,Amount,Month,Year
0,2023-01-27 10:10:12,Groceries,tesco stores 6601,6,1,2023
1,2023-01-27 11:50:32,Restaurants,zettle_*donovan?s bake,2,1,2023
2,2023-01-27 13:42:55,Services,apple.com/bill,4,1,2023
3,2023-01-28 09:59:20,Groceries,tesco stores 6601,5,1,2023
4,2023-01-28 11:48:36,Restaurants,zettle_*the good eatin,3,1,2023
5,2023-01-28 15:51:42,Groceries,toogoodtog mzs3m03xcn9,5,1,2023
6,2023-01-29 08:55:25,Entertainment,montys bar,21,1,2023
8,2023-01-29 09:49:29,Groceries,tesco stores 6601,9,1,2023
9,2023-01-29 10:08:56,Entertainment,urban 40,13,1,2023
10,2023-01-29 10:08:56,Entertainment,urban 40,6,1,2023


In [22]:
#check unassigned transactions and confirm all transactions are assigned to a category

unassigned = df.loc[df['Category'] == 'unassigned']
unassigned

,Date,Category,Description,Amount,Month,Year


In [25]:
# Get the latest month and year
latest_month = df['Month'].max()
latest_year = df['Year'].max()

# Filter the dataframe to include only transactions from the latest month
last_month_expenses = df[(df['Month'] == latest_month) & (df['Year'] == latest_year)]

In [26]:
last_month_expenses = last_month_expenses.groupby('Category')['Amount'].sum().reset_index()

last_month_expenses['Amount']=last_month_expenses['Amount'].astype('str')
last_month_expenses['Amount']=last_month_expenses['Amount'].str.replace('-','')
last_month_expenses['Amount']=last_month_expenses['Amount'].astype('float')        #get absolute figures

last_month_expenses = last_month_expenses[last_month_expenses["Category"].str.contains("Excluded|unassigned") == False]    #exclude "excluded" category
last_month_expenses = last_month_expenses.sort_values(by='Amount', ascending=False)    #sort values
last_month_expenses['Amount'] = last_month_expenses['Amount'].round().astype(int)      #round values

last_month_expenses

,Category,Amount
3,Transport,24
2,Restaurants,12
0,Coffee,8
1,Groceries,5


In [27]:
last_month_expenses_tot = last_month_expenses['Amount'].sum()
int(last_month_expenses_tot)

49

<h2>Create Top Banner for a summary of last month's income, recurring expenses, non-recurring expenses and savig</h2>

In [28]:
def calculate_difference(event):
    income = float(income_widget.value)
    recurring_expenses = float(recurring_expenses_widget.value)
    monthly_expenses = float(monthly_expenses_widget.value)
    difference = income - recurring_expenses - monthly_expenses
    difference_widget.value = str(difference)

income_widget = pn.widgets.TextInput(name="Income", value="0")
recurring_expenses_widget = pn.widgets.TextInput(name="Recurring Expenses", value="0")
monthly_expenses_widget = pn.widgets.TextInput(name="Non-Recurring Expenses", value=str(last_month_expenses_tot))
difference_widget = pn.widgets.TextInput(name="Last Month's Savings", value="0")

income_widget.param.watch(calculate_difference, "value")
recurring_expenses_widget.param.watch(calculate_difference, "value")
monthly_expenses_widget.param.watch(calculate_difference, "value")

pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget).show()

Launching server at http://localhost:59206


<h2>Create last month expense bar chart</h2>

In [29]:
last_month_expenses_chart = last_month_expenses.hvplot.bar(
    x='Category', 
    y='Amount', 
    height=250, 
    width=850, 
    title="Last Month Expenses",
    ylim=(0, 500))

last_month_expenses_chart

:Bars   [Category]   (Amount)

<h2>Create monthly expense trend bar chart</h2>

In [30]:
df['Date'] = pd.to_datetime(df['Date'])            # convert the 'Date' column to a datetime object
df['Month-Year'] = df['Date'].dt.to_period('M')    # extract the month and year from the 'Date' column and create a new column 'Month-Year'
monthly_expenses_trend_by_cat = df.groupby(['Month-Year', 'Category'])['Amount'].sum().reset_index()

monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].astype('str')
monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].str.replace('-','')
monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].astype('float')
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat["Category"].str.contains("Excluded") == False]

monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.sort_values(by='Amount', ascending=False)
monthly_expenses_trend_by_cat['Amount'] = monthly_expenses_trend_by_cat['Amount'].round().astype(int)
monthly_expenses_trend_by_cat['Month-Year'] = monthly_expenses_trend_by_cat['Month-Year'].astype(str)
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.rename(columns={'Amount': 'Amount '})

monthly_expenses_trend_by_cat

,Month-Year,Category,Amount
4,2023-01,Restaurants,45
1,2023-01,Entertainment,40
3,2023-01,Groceries,37
2,2023-01,Gifts,35
11,2023-02,Transport,24
7,2023-01,Transport,23
10,2023-02,Restaurants,12
8,2023-02,Coffee,8
0,2023-01,Coffee,7
9,2023-02,Groceries,5


In [31]:
#Define Panel widget

select_category1 = pn.widgets.Select(name='Select Category', options=[
    'All',
    'Self-Care',
    'Fines',
    'Study',
    'Coffee',
    'Groceries',
    'Shopping',
    'Restaurants',
    'Transport',
    'Travel',
    'Entertainment',
    'Gifts',
    'Services',
    #'Excluded'
])

select_category1

Select(options=['All', 'Self-Care', ...], value='All')

In [32]:
# define plot function
def plot_expenses(category):
    if category == 'All':
        plot_df = monthly_expenses_trend_by_cat.groupby('Month-Year').sum()
    else:
        plot_df = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat['Category'] == category].groupby('Month-Year').sum()
    plot = plot_df.hvplot.bar(x='Month-Year', y='Amount ')
    return plot

# define callback function
@pn.depends(select_category1.param.value)
def update_plot(category):
    plot = plot_expenses(category)
    return plot

# create layout
monthly_expenses_trend_by_cat_chart = pn.Row(select_category1, update_plot)
monthly_expenses_trend_by_cat_chart[1].width = 600

monthly_expenses_trend_by_cat_chart

Row
    [0] Select(options=['All', 'Self-Care', ...], value='All')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=600)

In [33]:
# ## Create summary table

df = df[['Date', 'Category', 'Description', 'Amount']]
df['Amount']=df['Amount'].astype('str')
df['Amount']=df['Amount'].str.replace('-','')
df['Amount']=df['Amount'].astype('float')        #get absolute figures

df = df[df["Category"].str.contains("Excluded") == False]    #exclude "excluded" category
df['Amount'] = df['Amount'].round().astype(int)      #round values
df

C:\Users\divya\AppData\Local\Temp\ipykernel_50812\3565075815.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Amount']=df['Amount'].astype('str')
C:\Users\divya\AppData\Local\Temp\ipykernel_50812\3565075815.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Amount']=df['Amount'].str.replace('-','')
C:\Users\divya\AppData\Local\Temp\ipykernel_50812\3565075815.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,Date,Category,Description,Amount
0,2023-01-27 10:10:12,Groceries,tesco stores 6601,6
1,2023-01-27 11:50:32,Restaurants,zettle_*donovan?s bake,2
2,2023-01-27 13:42:55,Services,apple.com/bill,4
3,2023-01-28 09:59:20,Groceries,tesco stores 6601,5
4,2023-01-28 11:48:36,Restaurants,zettle_*the good eatin,3
5,2023-01-28 15:51:42,Groceries,toogoodtog mzs3m03xcn9,5
6,2023-01-29 08:55:25,Entertainment,montys bar,21
8,2023-01-29 09:49:29,Groceries,tesco stores 6601,9
9,2023-01-29 10:08:56,Entertainment,urban 40,13
10,2023-01-29 10:08:56,Entertainment,urban 40,6


In [34]:
# Define a function to filter the dataframe based on the selected category
def filter_df(category):
    if category == 'All':
        return df
    return df[df['Category'] == category]

# Create a DataFrame widget that updates based on the category filter
summary_table = pn.widgets.DataFrame(filter_df('All'), height = 300,width=400)

# Define a callback that updates the dataframe widget when the category filter is changed
def update_summary_table(event):
    summary_table.value = filter_df(event.new)

# Add the callback function to the category widget
select_category1.param.watch(update_summary_table, 'value')

summary_table


DataFrame(height=300, sizing_mode='fixed', value=              ..., width=400)

In [39]:
# ## Create Final Dashboard

template = pn.template.FastListTemplate(
    title="Personal Finances Summary",
    sidebar=[
        pn.pane.Markdown("## *If you can't manage your money, making more won't help*"),
        pn.pane.PNG('image.png', sizing_mode='scale_both', width=300, height=300),
        pn.pane.Markdown(""),
        pn.pane.Markdown(""),
        select_category1
    ],
    main=[
        pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget, width=950),
        pn.Row(last_month_expenses_chart, height=240),
        pn.GridBox(
            monthly_expenses_trend_by_cat_chart[1],
            summary_table,
            ncols=2,
            width=500,  
            align='start',
            sizing_mode='stretch_width'
        )
    ]
)

template.show()

Launching server at http://localhost:59497
